In [1]:
! pip install datasets transformers seqeval
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import statistics as stats
from datasets import load_dataset, load_metric

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 524.9 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=5091b97fbfa49e1e7f56046e98082b9552fe7a2b4d9849be379247d12a4a6c19
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [3]:
ncbi = load_dataset("jnlpba")

Generating train split:   0%|          | 0/37094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7714 [00:00<?, ? examples/s]

Dataset jnlpba downloaded and prepared to /root/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
ncbi["train"][0]

{'id': '1',
 'tokens': ['IL-2',
  'gene',
  'expression',
  'and',
  'NF-kappa',
  'B',
  'activation',
  'through',
  'CD28',
  'requires',
  'reactive',
  'oxygen',
  'production',
  'by',
  '5-lipoxygenase',
  '.'],
 'ner_tags': [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]}

In [5]:
label_list = ncbi["train"].features[f"ner_tags"].feature.names
print(len(label_list))
label_list

11


['O',
 'B-DNA',
 'I-DNA',
 'B-RNA',
 'I-RNA',
 'B-cell_line',
 'I-cell_line',
 'B-cell_type',
 'I-cell_type',
 'B-protein',
 'I-protein']

In [6]:
MODEL_NAME = "AnonymousSub/fpdm_models_scibert_hybrid_epochs_4"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/427 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
label_all_tokens = True

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenize_and_align_labels(ncbi['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[102, 1902, 579, 170, 983, 940, 137, 5245, 579, 19685, 132, 2239, 833, 1389, 2557, 2978, 6608, 3858, 1865, 214, 305, 579, 2266, 18368, 205, 103], [102, 2239, 131, 111, 1389, 2557, 1437, 2629, 2315, 106, 1626, 1729, 163, 7655, 1261, 168, 105, 377, 2239, 2429, 121, 4006, 1865, 131, 12577, 579, 170, 145, 1902, 579, 170, 546, 137, 377, 4498, 205, 103], [102, 121, 1916, 105, 9242, 185, 405, 198, 1389, 2557, 16823, 3412, 147, 111, 2857, 5609, 2256, 131, 6608, 3858, 17753, 145, 25726, 546, 334, 220, 1761, 168, 1389, 2557, 579, 6224, 2239, 131, 111, 5245, 579, 19685, 132, 1352, 1389, 2557, 579, 14346, 1127, 137, 1902, 579, 170, 940, 205, 103], [102, 14316, 150, 131, 111, 1389, 2557, 3354, 11448, 241, 797, 147, 3862, 787, 9925, 4655, 1071, 422, 2594, 214, 111, 2239, 131, 26920, 106, 30132, 137, 305, 579, 2266, 18368, 205, 103], [102, 580, 453, 1739, 198, 2266, 18368, 9125, 9808, 16887, 2256, 334, 666, 5073, 1902, 579, 170, 940, 2168, 5245, 579, 19685, 132, 2239, 205, 103]], 'toke

In [11]:
tokenized_datasets = ncbi.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [12]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(label_list))

Downloading:   0%|          | 0.00/706 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at AnonymousSub/fpdm_models_scibert_hybrid_epochs_4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
args = TrainingArguments("SCIBERT-VOCAB-UNCASED-finetuned-NER",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps = 2,
    num_train_epochs=4,
    weight_decay=0.01
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [15]:
metric = load_metric("seqeval")

In [16]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 37094
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 4636


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.185200,0.264621,0.705324,0.818843,0.757856,0.919291
1,0.131300,0.297781,0.703232,0.816100,0.755474,0.917926
2,0.095900,0.326993,0.707896,0.818619,0.759242,0.917990
3,0.069800,0.352605,0.713525,0.817500,0.761982,0.919115


Saving model checkpoint to SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-500
Configuration saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-500/config.json
Model weights saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-500/pytorch_model.bin
tokenizer config file saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-500/tokenizer_config.json
Special tokens file saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-500/special_tokens_map.json
Saving model checkpoint to SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-1000
Configuration saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-1000/config.json
Model weights saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-1000/tokenizer_config.json
Special tokens file saved in SCIBERT-VOCAB-UNCASED-finetuned-NER/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set  don't have a correspon

TrainOutput(global_step=4636, training_loss=0.13221461532237305, metrics={'train_runtime': 1312.6229, 'train_samples_per_second': 113.038, 'train_steps_per_second': 3.532, 'total_flos': 5249115444000336.0, 'train_loss': 0.13221461532237305, 'epoch': 4.0})

In [19]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7714
  Batch size = 32


{'eval_loss': 0.35260501503944397,
 'eval_precision': 0.7135248705169549,
 'eval_recall': 0.8174998600459049,
 'eval_f1': 0.7619817892457407,
 'eval_accuracy': 0.9191150061058831,
 'eval_runtime': 29.7643,
 'eval_samples_per_second': 259.17,
 'eval_steps_per_second': 8.131,
 'epoch': 4.0}

In [20]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

KeyError: 'test'